# 📊 Opdracht 5: JSON Verwerking & App Bouwen

## 🎯 Wat gaan we doen?

Dit is het finale notebook! Je leert:
- AI's gestructureerde JSON laten maken
- JSON data verwerken en valideren
- Een complete applicatie bouwen met AI-generated data
- Alles wat je geleerd hebt combineren in één super app!

👉 Antwoordenblad: zie [antwoorden.md](antwoorden.md) voor voorbeeldantwoorden.

## 🏗️ Het Grote Plaatje

**Doel:** Maak een complete applicatie die AI gebruikt om gestructureerde data te maken en die data gebruikt in een echte app!

## 🧩 JSON in 5 minuten

JSON is een manier om data gestructureerd op te slaan als tekst.
- Object: {"key": value}
- Lijst: [item1, item2]
- Veelgebruikte types: string, number, boolean, array, object

Waarom handig?
- Makkelijk te lezen en te versturen via APIs
- Werkt direct met Python via de `json` module

Eerst even oefenen zonder AI:

In [ ]:
# JSON basics zonder AI
import json

# Python dict -> JSON string
persoon = {
    'naam': 'Sam',
    'leeftijd': 28,
    'hobbies': ['fietsen', 'koken']
}
json_string = json.dumps(persoon, ensure_ascii=False, indent=2)
print('Python -> JSON:')
print(json_string)

# JSON string -> Python dict
terug = json.loads(json_string)
print('\nJSON -> Python:')
print(terug['naam'], 'heeft', len(terug['hobbies']), 'hobbies')


In [ ]:
# Load environment variables from .env file
from dotenv import load_dotenv
import os

# Load the .env file from the same directory as the notebook
load_dotenv()

# Get configuration from environment variables
API_KEY = os.getenv('API_KEY', 'ollama')
BASE_URL = os.getenv('BASE_URL', 'http://localhost:11434/v1')
MODEL = os.getenv('MODEL', 'llama2')

print(f"🔗 Configuratie: {BASE_URL}")
print(f"🔑 API Key: {'*' * 8 + API_KEY[-4:] if API_KEY and len(API_KEY) > 4 else 'Niet ingesteld'}")
print(f"🤖 Model: {MODEL}")


In [ ]:
from openai import OpenAI

# Initialise de AI client
client = OpenAI(
    api_key=API_KEY,
    base_url=BASE_URL
)


## 📋 JSON Basics: AI als Data Generator

Eerst leren we AI's om perfect geformatteerde JSON te maken:

In [ ]:
import json
import re


def generate_structured_data(prompt, json_schema):
    """Laat AI gestructureerde JSON data maken"""
    
    system_message = (
        "Je bent een data generator. Genereer ALLEEN geldige JSON volgens het gegeven schema. "
        "Geef geen extra tekst, alleen de JSON. Zorg dat alle velden ingevuld zijn."
    )
    
    full_prompt = f"{prompt}\n\nSchema:\n{json_schema}\n\nGenereer JSON:"
    
    try:
        response = client.chat.completions.create(
            model=MODEL,
            messages=[
                {"role": "system", "content": system_message},
                {"role": "user", "content": full_prompt}
            ],
            max_tokens=300,
            temperature=0.7
        )
        
        # Probeer JSON te extraheren
        raw_response = response.choices[0].message.content
        json_data = extract_json_from_text(raw_response)
        
        return json_data
        
    except Exception as e:
        return {"error": f"Could not generate data: {e}"}

def extract_json_from_text(text):
    """Probeer JSON uit tekst te halen (ook tussen ```json blokken)"""
    import re, json as _json
    # 1) Kijk naar fenced code blocks ```json ... ```
    fence = re.search(r'```json\s*(.*?)```', text, re.DOTALL|re.IGNORECASE)
    if fence:
        chunk = fence.group(1).strip()
        try:
            return _json.loads(chunk)
        except _json.JSONDecodeError:
            pass
    # 2) Probeer volledige tekst
    try:
        return _json.loads(text.strip())
    except _json.JSONDecodeError:
        pass
    # 3) Zoek eerste JSON object/array
    for pattern in [r'\{.*?\}', r'\[.*?\]']:
        matches = re.findall(pattern, text, re.DOTALL)
        for m in matches:
            try:
                return _json.loads(m)
            except _json.JSONDecodeError:
                continue
    return {"error": "No valid JSON found", "raw": text[:500]}

# Test basis JSON generatie
person_schema = """{{
  "name": "string",
  "age": "number",
  "city": "string",
  "hobbies": ["array", "of", "strings"],
  "occupation": "string"
}}"""

result = generate_structured_data(
    "Maak een profiel van een fictional persoon uit Nederland",
    person_schema
)

print("🧪 TEST RESULTAAT:")
print(json.dumps(result, indent=2, ensure_ascii=False))


## 🎨 JSON Data Validator & Processor

Nu maken we tools om JSON data te valideren en verwerken:

In [ ]:
import datetime
class JSONProcessor:
    def __init__(self):
        self.processed_data = []
        self.schemas = {}
    
    def add_schema(self, name, schema):
        """Voeg een JSON schema toe"""
        self.schemas[name] = schema
        print(f"📋 Schema '{name}' toegevoegd")
    
    def validate_json(self, data, schema_name=None):
        """Valideer JSON data tegen een schema"""
        if isinstance(data, str):
            try:
                data = json.loads(data)
            except json.JSONDecodeError:
                return False, "Invalid JSON format"
        
        if schema_name and schema_name in self.schemas:
            schema = self.schemas[schema_name]
            # Simpele schema validatie
            try:
                schema_obj = json.loads(schema) if isinstance(schema, str) else schema
                for key in schema_obj.keys():
                    if key not in data:
                        return False, f"Missing field: {key}"
                return True, "Valid JSON"
            except:
                return True, "Schema validation skipped"  # Fallback
        
        return True, "Valid JSON"
    
    def process_data(self, data, processor_func=None):
        """Verwerk JSON data met een custom functie"""
        if processor_func:
            try:
                processed = processor_func(data)
                self.processed_data.append({
                    "original": data,
                    "processed": processed,
                    "timestamp": datetime.datetime.now()
                })
                return processed
            except Exception as e:
                return {"error": f"Processing failed: {e}"}
        
        return data
    
    def generate_and_process(self, prompt, schema_name, processor_func=None):
        """Complete pipeline: genereren, valideren, verwerken"""
        
        # 1. Genereer data
        schema = self.schemas.get(schema_name, "{}")
        raw_data = generate_structured_data(prompt, schema)
        
        if "error" in raw_data:
            return raw_data
        
        # 2. Valideer data
        is_valid, message = self.validate_json(raw_data, schema_name)
        
        if not is_valid:
            return {"error": f"Validation failed: {message}"}
        
        # 3. Verwerk data
        if processor_func:
            processed = self.process_data(raw_data, processor_func)
            return {
                "original": raw_data,
                "processed": processed,
                "validation": message
            }
        
        return {
            "data": raw_data,
            "validation": message
        }

# Setup processor
processor = JSONProcessor()

# Voeg schemas toe
processor.add_schema("person", person_schema)

processor.add_schema("product", """{
  "name": "string",
  "price": "number",
  "category": "string",
  "description": "string",
  "rating": "number (1-5)",
  "in_stock": "boolean"
}""")

processor.add_schema("event", """{
  "title": "string",
  "date": "string (YYYY-MM-DD)",
  "time": "string (HH:MM)",
  "location": "string",
  "attendees": "number",
  "description": "string"
}""")

print("\n✅ JSON Processor ready!")


## 🧪 Experimenteer met Data Generatie

Nu gaan we verschillende soorten data genereren:

In [ ]:
# Test verschillende data types

def format_person_data(data):
    """Maak persoon data mooi leesbaar"""
    return f"👤 {data['name']} ({data['age']}) uit {data['city']} - {data['occupation']}\nHobbies: {', '.join(data['hobbies'])}"

def calculate_product_value(data):
    """Bereken product waarde score"""
    value_score = (data['rating'] * 20) - (data['price'] / 10)
    data['value_score'] = round(value_score, 2)
    data['recommendation'] = "Aanrader!" if value_score > 50 else "Gemiddeld" if value_score > 20 else "Overslaan"
    return data

print("🧪 EXPERIMENT 1: Personen genereren")
print("=" * 40)

for i in range(3):
    result = processor.generate_and_process(
        f"Maak een interessant persoon profiel #{i+1} met unieke hobbies",
        "person",
        format_person_data
    )
    
    if "processed" in result:
        print(f"\n{result['processed']}")
    else:
        print(f"\nError: {result}")

print("\n\n🛍️ EXPERIMENT 2: Producten met waardering")
print("=" * 40)

product_prompts = [
    "Een cool tech gadget",
    "Een eco-vriendelijk huishoudproduct", 
    "Een luxe fashion item"
]

for prompt in product_prompts:
    result = processor.generate_and_process(
        f"Maak een {prompt} met realistische prijs en beschrijving",
        "product",
        calculate_product_value
    )
    
    if "processed" in result:
        product = result['processed']
        print(f"\n🛒 {product['name']} - €{product['price']}")
        print(f"   ⭐ {product['rating']}/5 | 💰 Value: {product['value_score']} | {product['recommendation']}")
        print(f"   📦 {product['description']}")
    else:
        print(f"\nError: {result}")

## 💡 EXPERIMENTEER TIJD!

**JOUW BEURT:** Maak jouw eigen data schemas en processors!

In [ ]:
# JOUW EXPERIMENTEER RUIMTE!
# Ideeën voor eigen schemas:
# - Restaurant reviews
# - Movie/book recommendations  
# - Travel destinations
# - Recipe generator
# - Job postings
# - Social media posts

# Voorbeeld: Restaurant schema
restaurant_schema = """{
  "name": "string",
  "cuisine_type": "string",
  "price_range": "string (€, €€, €€€)",
  "rating": "number (1-5)",
  "signature_dish": "string",
  "atmosphere": "string",
  "location": "string"
}"""

processor.add_schema("restaurant", restaurant_schema)

def format_restaurant_review(data):
    """Format restaurant data as review"""
    stars = "⭐" * int(data['rating'])
    return f"🍽️ {data['name']} ({data['cuisine_type']})\n{stars} {data['rating']}/5 | {data['price_range']} | {data['location']}\n🍴 Probeer: {data['signature_dish']}\n🏡 Sfeer: {data['atmosphere']}"

# TODO: Voeg jouw eigen schema en processor toe!
# jouw_schema = """{
#   "field1": "type",
#   "field2": "type"
# }"""
# processor.add_schema("jouw_type", jouw_schema)

# def jouw_processor(data):
#     # Verwerk je data hier
#     return data

# Test restaurant generator
print("🍽️ RESTAURANT GENERATOR TEST:")
result = processor.generate_and_process(
    "Maak een gezellig Italiaans restaurant in Amsterdam",
    "restaurant",
    format_restaurant_review
)

if "processed" in result:
    print(result['processed'])
else:
    print(f"Error: {result}")

# Test jouw eigen schema hier:

## 📝 Jouw Opdracht

We kiezen standaard voor de Restaurant Generator. Klaar? Dan kun je dit kopiëren voor andere ideeën.

Wat je gaat doen (5 kleine stappen):
1) Schema maken (welke velden wil je?)
2) JSON genereren met AI
3) Controleren of de JSON klopt
4) Verwerken (klein beetje slimme logica)
5) Mooi printen (3–5 regels)

Checklist:
- [ ] Geldige JSON (geen extra tekst)
- [ ] Alle velden aanwezig
- [ ] Simpele processor (score/categorie/aanbeveling)
- [ ] Netjes geprint
- [ ] (Bonus) Meerdere voorbeelden vergelijken


### 🚀 Start hier (Restaurant Generator)
Vul eventueel stad/keuken aan. Run de cellen van boven naar beneden.


In [ ]:
# Stap 1: Schema (simpel houden: 5–6 velden)
restaurant_schema_min = """{
  \"name\": \"string\",
  \"cuisine_type\": \"string\",
  \"city\": \"string\",
  \"price_per_person\": \"number\",
  \"rating\": \"number (1-5)\",
  \"signature_dish\": \"string\"
}"""

# Vereiste velden en verwachte types voor eenvoudige check
required_fields = ['name', 'cuisine_type', 'city', 'price_per_person', 'rating', 'signature_dish']
expected_types = {
    'name': str,
    'cuisine_type': str,
    'city': str,
    'price_per_person': (int, float),
    'rating': (int, float),
    'signature_dish': str
}
print('✅ Schema klaar (restaurant_schema_min)')


In [ ]:
# Stap 2: JSON genereren met AI
prompt_restaurant = 'Maak een gezellig Italiaans restaurant in Utrecht'
gen1 = generate_structured_data(prompt_restaurant, restaurant_schema_min)
import json as _json
print('🧾 AI JSON:')
print(_json.dumps(gen1, indent=2, ensure_ascii=False))


In [ ]:
# Stap 3: Controleren (zit alles erin en klopt het type?)
def validate_required_fields(d, required, types=None):
    if not isinstance(d, dict):
        return False, required, [('GEEN_OBJECT', type(d).__name__, 'dict')]
    missing = [k for k in required if k not in d]
    wrong = []
    if types:
        for k, t in types.items():
            if k in d and not isinstance(d[k], t):
                wrong.append((k, type(d[k]).__name__, getattr(t, '__name__', str(t))))
    ok = not missing and not wrong
    return ok, missing, wrong

ok, missing, wrong = validate_required_fields(gen1, required_fields, expected_types)
if ok:
    print('✅ JSON ziet er goed uit!')
else:
    print('❌ JSON mist of heeft verkeerde velden:')
    if missing: print('  Ontbrekend:', missing)
    if wrong: print('  Verkeerd type:', wrong)


In [ ]:
# Stap 4: Verwerken (voeg kleine logica en print mooi)
def simple_restaurant_processor(d):
    d = dict(d)
    price = d.get('price_per_person', 25)
    if price < 20: cat, icon = 'Budget-vriendelijk', '💰'
    elif price < 40: cat, icon = 'Gemiddeld', '💳'
    else: cat, icon = 'Luxe', '💎'
    d['price_category'] = cat
    d['price_emoji'] = icon
    rating = float(d.get('rating', 3))
    if rating >= 4.5: d['recommendation'] = 'Zeer aanbevolen! ⭐⭐⭐'
    elif rating >= 4.0: d['recommendation'] = 'Aanrader! ⭐⭐'
    else: d['recommendation'] = 'Oké optie ⭐'
    return d

if ok:
    out = simple_restaurant_processor(gen1)
    print('Resultaat (kort en krachtig):')
    print(f"🍽️ {out['name']} ({out['cuisine_type']}, {out['city']})")
    print(f"   {out['price_emoji']} {out['price_category']} (± €{out['price_per_person']}/p) | ⭐ {out['rating']}/5")
    print(f"   🍴 Probeer: {out['signature_dish']}")
    print(f"   {out['recommendation']}")
else:
    print('Fix de JSON in stap 2 of schema in stap 1')


### ⭐ Bonus (optioneel)
Genereer 3 restaurants en vergelijk ze op prijs en rating.


In [ ]:
cities = ['Utrecht', 'Amsterdam', 'Groningen']
items = []
for c in cities:
    g = generate_structured_data(f'Maak een Italiaans restaurant in {c}', restaurant_schema_min)
    ok2, miss2, wrong2 = validate_required_fields(g, required_fields, expected_types)
    if ok2:
        items.append(simple_restaurant_processor(g))

print('Overzicht:')
for it in items:
    print(f"- {it['name']} ({it['city']}): ⭐ {it['rating']}/5, €{it['price_per_person']}/p, {it['price_category']}")


In [ ]:
# 🏗️ JOUW DATA GENERATOR APP - STAP VOOR STAP!

# 📚 STAP 1: BASIS SETUP (Iedereen start hier!)

# Eerst importeren we wat we nodig hebben
import json
import datetime
from openai import OpenAI
import os

# Setup client (gebruik configuratie uit eerdere cell)
client = OpenAI(api_key=API_KEY, base_url=BASE_URL)

print("🎯 STAP 1: BASIS SETUP")
print("=" * 30)

# Voorbeeld: Restaurant Schema (simpel!)
restaurant_schema = """{
  "name": "string",
  "cuisine_type": "string", 
  "city": "string",
  "price_per_person": "number",
  "rating": "number (1-5)",
  "signature_dish": "string",
  "atmosphere": "string"
}"""

# Test AI data generatie
print("🧪 Test: Genereer een restaurant")
test_restaurant = generate_structured_data(
    "Maak een gezellig Italiaans restaurant in Amsterdam", 
    restaurant_schema
)

print("Resultaat:")
print(json.dumps(test_restaurant, indent=2, ensure_ascii=False))

print("\n" + "="*50)
print("🎯 STAP 2: MAAK HET SLIMMER")
print("=" * 30)

# Voeg een data processor toe
def process_restaurant_data(restaurant_data):
    """Maak restaurant data slimmer en leesbaarder"""
    
    # Voeg extra informatie toe
    price = restaurant_data.get('price_per_person', 25)
    
    # Bepaal prijsklasse
    if price < 20:
        restaurant_data['price_category'] = "Budget-vriendelijk"
        restaurant_data['price_emoji'] = "💰"
    elif price < 40:
        restaurant_data['price_category'] = "Gemiddeld" 
        restaurant_data['price_emoji'] = "💳"
    else:
        restaurant_data['price_category'] = "Luxe"
        restaurant_data['price_emoji'] = "💎"
    
    # Voeg aanbeveling toe
    rating = restaurant_data.get('rating', 3)
    if rating >= 4.5:
        restaurant_data['recommendation'] = "Zeer aanbevolen! ⭐⭐⭐"
    elif rating >= 4.0:
        restaurant_data['recommendation'] = "Aanrader! ⭐⭐"
    else:
        restaurant_data['recommendation'] = "Oké optie ⭐"
    
    return restaurant_data

# Test de processor
print("🧪 Test: Verwerk restaurant data")
processed = process_restaurant_data(test_restaurant.copy())
print(f"🍽️ {processed['name']}")
print(f"   {processed['price_emoji']} {processed['price_category']} (€{processed['price_per_person']})")
print(f"   {processed['recommendation']}")
print(f"   🍴 Probeer: {processed['signature_dish']}")

print("\n" + "="*50)
print("🎯 STAP 3: BOUW JOUW APP!")
print("=" * 30)

# Template voor jouw app
class MijnDataGeneratorApp:
    def __init__(self, app_type):
        self.app_type = app_type
        self.generated_items = []
        self.schema = {}
        
        print(f"🚀 {app_type} App gestart!")
    
    def set_schema(self, schema):
        """Stel je data schema in"""
        self.schema = schema
        print("📋 Schema ingesteld!")
    
    def generate_item(self, prompt):
        """Genereer een item met AI"""
        try:
            # Gebruik de generate_structured_data functie
            result = generate_structured_data(prompt, self.schema)
            
            if 'error' not in result:
                self.generated_items.append({
                    'data': result,
                    'created_at': datetime.datetime.now().strftime("%H:%M:%S"),
                    'prompt': prompt
                })
                return result
            else:
                return {"error": "AI generatie mislukt"}
                
        except Exception as e:
            return {"error": f"Fout: {str(e)}"}
    
    def process_item(self, item, processor_function):
        """Verwerk een item met jouw functie"""
        try:
            return processor_function(item)
        except Exception as e:
            return {"error": f"Processing fout: {str(e)}"}
    
    def show_all_items(self):
        """Toon alle gegenereerde items"""
        print(f"\n📊 {len(self.generated_items)} items gegenereerd:")
        for i, item in enumerate(self.generated_items, 1):
            print(f"{i}. Gemaakt om {item['created_at']}")
            print(f"   Prompt: {item['prompt']}")
    
    def save_to_file(self, filename):
        """Sla alle data op naar een file in ./generated-json/"""

        # Zorg dat de map bestaat
        folder = "generated-json"
        os.makedirs(folder, exist_ok=True)
        filepath = os.path.join(folder, filename)

        try:
            data = {
                'app_type': self.app_type,
                'total_items': len(self.generated_items),
                'items': self.generated_items,
                'saved_at': datetime.datetime.now().isoformat()
            }
            
            with open(filepath, 'w', encoding='utf-8') as f:
                json.dump(data, f, indent=2, ensure_ascii=False)
            
            return f"✅ Opgeslagen naar {filepath}"
        except Exception as e:
            return f"❌ Opslaan mislukt: {str(e)}"

# 🧪 TEST DE APP TEMPLATE
print("🧪 Test: App template")
test_app = MijnDataGeneratorApp("Restaurant Generator")
test_app.set_schema(restaurant_schema)

# Genereer een item
result = test_app.generate_item("Maak een modern sushi restaurant in Rotterdam")
if 'error' not in result:
    print(f"✅ Gegenereerd: {result['name']}")
else:
    print(f"❌ Fout: {result['error']}")

test_app.show_all_items()
test_app.save_to_file("restaurants.json")

print("\n" + "="*60)
print("🎯 NU BEN JIJ AAN DE BEURT!")
print("Kies één van de 4 opties en vul hieronder in:")

# 🍽️ OPTIE 1: RESTAURANT GENERATOR
print("\n🍽️ OPTIE 1: Restaurant Generator")
print("Uncomment deze code als je restaurants wilt genereren:")

# restaurant_app = MijnDataGeneratorApp("Restaurant Generator")
# restaurant_app.set_schema(restaurant_schema)
# 
# # Test verschillende restaurants
# cities = ["Amsterdam", "Utrecht", "Groningen"]
# cuisines = ["Italiaans", "Thais", "Frans"]
# 
# for city in cities:
#     for cuisine in cuisines[:1]:  # Alleen eerste cuisine om kort te houden
#         prompt = f"Maak een {cuisine} restaurant in {city}"
#         result = restaurant_app.generate_item(prompt)
#         if 'error' not in result:
#             processed = process_restaurant_data(result)
#             print(f"🍽️ {processed['name']} ({city})")
#             print(f"   {processed['recommendation']}")

# 🎬 OPTIE 2: MOVIE NIGHT PLANNER  
print("\n🎬 OPTIE 2: Movie Night Planner")
print("TODO: Maak jouw schema en app hier!")

# movie_schema = """{
#   "movie_title": "string",
#   "genre": "string", 
#   "duration_minutes": "number",
#   "mood": "string",
#   "snacks": ["array", "of", "strings"],
#   "rating": "number (1-10)"
# }"""

# 🎵 OPTIE 3: PLAYLIST CREATOR
print("\n🎵 OPTIE 3: Playlist Creator") 
print("TODO: Maak jouw schema en app hier!")

# 🏃 OPTIE 4: WORKOUT BUILDER
print("\n🏃 OPTIE 4: Workout Builder")
print("TODO: Maak jouw schema en app hier!")

print("\n💡 TIPS VOOR BEGINNERS:")
print("1. Kies 1 optie die je leuk vindt")
print("2. Copy-paste de restaurant code als template")
print("3. Verander de schema voor jouw data type")
print("4. Test kleine stapjes - dan zie je wat werkt!")
print("5. Vraag om hulp als je vastloopt!")

print("\n🏆 BONUS: Als je klaar bent:")
print("- Voeg meer velden toe aan je schema")
print("- Maak je processor functie slimmer") 
print("- Genereer meerdere items en vergelijk ze")
print("- Sla je data op naar een file")

In [ ]:
# Bouw hier je applicatie

## ✅ Check Jezelf

Kun je deze vragen beantwoorden?

1. **Hoe maak je AI gestructureerde JSON laten genereren?**
2. **Waarom is data validatie belangrijk bij AI-generated content?**
3. **Hoe kun je alle workshop concepten combineren in één applicatie?**
4. **Wat zijn de voordelen van JSON voor AI applications?**

Schrijf je antwoorden hieronder:

**Mijn antwoorden:**

1. Gestructureerde JSON generatie: 
2. Data validatie: 
3. Concepten combineren: 
4. JSON voordelen: 

## 🎉 GEFELICITEERD!

Je hebt alle workshop onderdelen voltooid! Je kunt nu:

✅ **API Calls maken** - Verbinden met verschillende AI providers
✅ **Geheugen toevoegen** - Chatbots met context en persoonlijkheid
✅ **AI Conversaties** - Meerdere AI's met elkaar laten praten
✅ **Function Calling** - AI's tools geven om acties uit te voeren
✅ **JSON Verwerking** - Gestructureerde data genereren en verwerken

## 🚀 Wat nu?

**Volgende stappen:**
- Bouw je eigen AI-powered applicatie
- Experimenteer met verschillende API providers
- Combineer concepten op creatieve manieren
- Deel je creaties met anderen!

---

## 💡 Pro Tips voor Thuis

- **Start klein en bouw uit** - Begin met simpele features
- **Experimenteer met prompts** - Kleine wijzigingen kunnen grote verschillen maken
- **Error handling is essentieel** - AI's zijn krachtig maar niet perfect
- **Bewaar interessante resultaten** - Bouw een bibliotheek van goede prompts
- **Combineer met andere tools** - Databases, web frameworks, etc.
- **Deel en leer van anderen** - De AI community is enorm behulpzaam!

## 🎓 Je bent nu een AI API Expert!

Veel succes met je AI projecten! 🌟